In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from service.utils.spark import get_spark_session
# spark = SparkSession.builder.getOrCreate()
spark = get_spark_session(dev=True)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/23 14:54:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
test_namespace = 'gold' 
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {test_namespace}").show()

++
||
++
++



In [3]:
payment_df = spark.read.csv("s3a://warehousedev/bronze/tsv/payment.tsv", header=True, sep='\t')

25/09/23 14:55:00 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [4]:
order_customer_id = payment_df.select('order_id', 'customer_id').distinct()

In [5]:
order_status_df = spark.read.csv("s3a://warehousedev/bronze/tsv/order_status.tsv", header=True, sep='\t')

In [6]:
delivered_order_id = order_status_df.filter(F.col('status') == 'delivered_customer').select('order_id')

In [7]:
delivered_order_status = order_status_df.join(delivered_order_id, on='order_id', how='inner')

In [8]:
format_string = 'yyyy-MM-dd HH:mm:ss'
delivered_order_status = delivered_order_status.withColumn('timestamp_clean', F.substring(F.col('timestamp'), 1, 19)) \
                                .withColumn('timestamp', F.to_timestamp('timestamp_clean', 'yyyy-MM-dd HH:mm:ss')) \
                                .drop('timestamp_clean')
delivered_order_status.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- status: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [9]:
distinct_status = delivered_order_status.select('status').distinct().collect()
pivot_values = [row.status for row in distinct_status]

In [10]:
pivoted_order_status_df = delivered_order_status.groupBy('order_id') \
    .pivot('status', pivot_values) \
    .agg(F.first('timestamp'))


In [11]:
pivoted_order_customer_df = pivoted_order_status_df.join(order_customer_id, on='order_id', how='inner')

In [12]:
from functools import reduce

# stauts가 하나라도 null인 row 확인. 격리 필요
conditions = [F.isnull(F.col(c)) for c in pivoted_order_customer_df.columns]
final_condition = reduce(lambda a, b: a | b, conditions)
rows_with_any_na = pivoted_order_customer_df.filter(final_condition)
rows_with_any_na.count()

16

In [13]:
tmp_order_timestamp = pivoted_order_customer_df.select('order_id', 'customer_id', 'purchase', 'approved', 'delivered_carrier', 'delivered_customer').dropna()

In [14]:
tmp_order_timestamp.writeTo(f"{test_namespace}.tmp_order_timestamp").createOrReplace()

In [15]:
tmp_order_timestamp.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- purchase: timestamp (nullable = true)
 |-- approved: timestamp (nullable = true)
 |-- delivered_carrier: timestamp (nullable = true)
 |-- delivered_customer: timestamp (nullable = true)



In [16]:
spark.stop()